# Setup

This section covers the initial setup, including library imports, path definitions, and mounting Google Drive.

## Mount Google Drive

In [6]:
# from google.colab import drive
# drive.mount('/drive', force_remount=True)

## Import Libraries

In [7]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
# !pip install openpyxl
import openpyxl
# !pip install holidays
import holidays
from datetime import datetime

# Enhanced ML imports for better preprocessing pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin

warnings.filterwarnings('ignore')

## Define Paths

In [ ]:
# Path variables
# base_dir = "/drive/MyDrive/Data Mining and Machine Learning/Progetto"
base_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))), "JupyterOutputs")
cleaned_integrated_dir = os.path.join(base_dir, "DataIntegrated")
cleaned_integrated_data_file = os.path.join(cleaned_integrated_dir, "cleaned_integrated_crime_data.csv")
feature_engineering_dir = os.path.join(base_dir, "FeatureEngineered")
feature_engineered_file_path = os.path.join(feature_engineering_dir, "feature_engineered_crime_data.csv")
pd_codes_file = os.path.join(os.path.dirname(os.path.dirname(base_dir)), "Documents", "PDCode_PenalLaw.xlsx")

# Check if pd_codes_file exists
if not os.path.exists(pd_codes_file):
    raise FileNotFoundError(f"PD codes file not found at: {pd_codes_file}")

# Ensure output directory exists
os.makedirs(feature_engineering_dir, exist_ok=True)

print(f"Base directory: {base_dir}")
print(f"Feature engineering output directory: {feature_engineering_dir}")
print(f"Looking for cleaned integrated data at: {cleaned_integrated_data_file}")
print(f"Looking for PD codes file at: {pd_codes_file}")

Base directory: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs
Feature engineering output directory: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\FeatureEngineered
Looking for cleaned integrated data at: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\DataIntegrated\cleaned_integrated_crime_data.csv
Looking for PD codes file at: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\..\Documents\PDCode_PenalLaw.xlsx


# Load Integrated Data

Load the dataset produced by the Data Integration phase.

In [9]:
# Load dataset
print("=== Loading Integrated Data ===")
try:
    if os.path.exists(cleaned_integrated_data_file):
        df = pd.read_csv(cleaned_integrated_data_file)
        initial_rows = len(df)
        print(f"Dataset loaded successfully: {initial_rows} rows and {df.shape[1]} columns")
        print(f"Columns in the dataset: {df.columns.tolist()}")

        # Basic validation
        if initial_rows == 0:
            raise ValueError("Dataset is empty")
        if df.shape[1] < 5:
            raise ValueError("Dataset has insufficient columns")

    else:
        raise FileNotFoundError(f"Could not find cleaned integrated dataset at: {cleaned_integrated_data_file}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise RuntimeError(f"Failed to load required dataset for feature engineering: {e}")

=== Loading Integrated Data ===
Dataset loaded successfully: 2496759 rows and 34 columns
Columns in the dataset: ['BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE']


## Initial Data Overview

Display basic information, summary statistics, and a sample of the loaded dataset.

In [10]:
# Display basic dataset overview
print("\n=== Dataset Overview ===")
print(df.info())
print("\n=== Summary Statistics ===")
print(df.describe())
print("\n=== Sample Row ===")
print(df.sample())


=== Dataset Overview ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2496759 entries, 0 to 2496758
Data columns (total 34 columns):
 #   Column              Dtype  
---  ------              -----  
 0   BORO_NM             object 
 1   CMPLNT_FR_DT        object 
 2   CMPLNT_FR_TM        object 
 3   KY_CD               int64  
 4   LAW_CAT_CD          object 
 5   LOC_OF_OCCUR_DESC   object 
 6   OFNS_DESC           object 
 7   PARKS_NM            object 
 8   PD_CD               int64  
 9   PREM_TYP_DESC       object 
 10  SUSP_AGE_GROUP      object 
 11  SUSP_RACE           object 
 12  SUSP_SEX            object 
 13  VIC_AGE_GROUP       object 
 14  VIC_RACE            object 
 15  VIC_SEX             object 
 16  Latitude            float64
 17  Longitude           float64
 18  BAR_DISTANCE        float64
 19  NIGHTCLUB_DISTANCE  float64
 20  ATM_DISTANCE        float64
 21  ATMS_COUNT          float64
 22  BARS_COUNT          float64
 23  BUS_STOPS_COUNT     float64
 24

# Load External PD Codes

Load the external Excel file containing mappings between PD codes and descriptions.

In [11]:
print("\n=== Loading External PD Codes ===")
try:
    if os.path.exists(pd_codes_file):
        df_codes = pd.read_excel(pd_codes_file)
        initial_rows = len(df_codes)
        print(f"PD Codes dataset loaded successfully: {initial_rows} rows and {df_codes.shape[1]} columns")
        print(f"Columns in the codes dataset: {df_codes.columns.tolist()}")
    else:
        raise FileNotFoundError(f"Could not find PD codes dataset at: {pd_codes_file}")
except Exception as e:
    print(f"Error loading PD codes dataset: {e}")
    # Instead of exit(1), print the error and continue
    print(f"Error details: {e}")


=== Loading External PD Codes ===
PD Codes dataset loaded successfully: 4671 rows and 5 columns
Columns in the codes dataset: ['PDCODE_VALUE', 'LAW_NYS', 'CATEGORY', 'LIT_LONG', 'LIT_SHORT']


# Feature Engineering

This section focuses on creating new features and refining existing ones based on domain knowledge and data characteristics.

## 1. Impute Missing `OFNS_DESC`

Fill missing offense descriptions (`OFNS_DESC`) using the mapping from `PD_CD` to `LIT_SHORT` found in the external codes file.

In [12]:
print("\n=== Feature Engineering: Imputing OFNS_DESC ===")
# Ensure both codes are of type string
df['PD_CD'] = df['PD_CD'].astype(str)
df_codes['PDCODE_VALUE'] = df_codes['PDCODE_VALUE'].astype(str)

# Create a mapping dictionary from the first occurrence of each code
code_to_lit_short = df_codes.drop_duplicates(subset='PDCODE_VALUE').set_index('PDCODE_VALUE')['LIT_SHORT'].to_dict()

# Fill OFNS_DESC where it's '(null)' using the mapping
null_mask = df['OFNS_DESC'] == '(null)'
initial_nulls = df['OFNS_DESC'].isin(['(null)', None, np.nan]).sum()
df.loc[null_mask, 'OFNS_DESC'] = (
    df.loc[null_mask, 'PD_CD'].map(code_to_lit_short)
    .str.strip()
    .fillna(df.loc[null_mask, 'OFNS_DESC']) # Keep original if map fails
)

# count missing after imputation
final_nulls = df['OFNS_DESC'].isin(['(null)', None, np.nan]).sum()

# print summary
print(f"OFNS_DESC missing before imputation: {initial_nulls}")
print(f"OFNS_DESC missing after imputation:  {final_nulls}")
print(f"Number of values filled: {initial_nulls - final_nulls}")


=== Feature Engineering: Imputing OFNS_DESC ===
OFNS_DESC missing before imputation: 44
OFNS_DESC missing after imputation:  0
Number of values filled: 44


## 2. Create Temporal Features

Extract time-based features from the complaint date and time:
- `HOUR`: Hour of the day (0-23)
- `WEEKDAY`: Name of the day (e.g., MONDAY)
- `IS_WEEKEND`: Binary flag (1 for Saturday/Sunday, 0 otherwise)
- `MONTH`: Month of the year (1-12)
- `SEASON`: Categorical season (WINTER, SPRING, SUMMER, AUTUMN)
- `TIME_BUCKET`: Categorical time of day (NIGHT, MORNING, AFTERNOON, EVENING)

In [13]:
print("\n=== Feature Engineering: Temporal Features ===")
# 1) Unified timestamp (temporary)
df['TIMESTAMP'] = pd.to_datetime(
    df['CMPLNT_FR_DT'] + ' ' + df['CMPLNT_FR_TM'],
    format='%Y/%m/%d %H:%M:%S.%f', # Adjusted format based on previous steps
    errors='coerce'
)

# Handle potential coercion errors (if any)
invalid_timestamps = df['TIMESTAMP'].isna().sum()
if invalid_timestamps > 0:
    print(f"Warning: {invalid_timestamps} rows had invalid date/time formats and resulted in NaT timestamps.")

# 2) Hour of the day
df['HOUR'] = df['TIMESTAMP'].dt.hour

# 3) Day
df['DAY'] = df['TIMESTAMP'].dt.day

# 4) Weekday
df['WEEKDAY'] = df['TIMESTAMP'].dt.day_name().str.upper()

# 5) Flag weekend (1 = Saturday/Sunday)
df['IS_WEEKEND'] = df['WEEKDAY'].isin(['SATURDAY', 'SUNDAY']).astype(int)

# 6) Month
df['MONTH'] = df['TIMESTAMP'].dt.month

# 7) Year
df['YEAR'] = df['TIMESTAMP'].dt.year

# 8) Season
def map_season(month):
    if pd.isna(month): return 'UNKNOWN' # Handle potential NaNs from NaT timestamps
    if month in [12, 1, 2]: return 'WINTER'
    elif month in [3, 4, 5]: return 'SPRING'
    elif month in [6, 7, 8]: return 'SUMMER'
    else: return 'AUTUMN'
df['SEASON'] = df['MONTH'].apply(map_season)

# 9) Time Bucket
def time_bucket(hour):
    if pd.isna(hour): return 'UNKNOWN' # Handle potential NaNs
    if hour < 6: return 'NIGHT'
    elif hour < 12: return 'MORNING'
    elif hour < 18: return 'AFTERNOON'
    else: return 'EVENING'
df['TIME_BUCKET'] = df['HOUR'].apply(time_bucket)

# Drop intermediate and original time columns
df = df.drop(columns=['CMPLNT_FR_TM', 'CMPLNT_FR_DT', 'TIMESTAMP'])
print("Created temporal features: HOUR, DAY, WEEKDAY, IS_WEEKEND, MONTH, YEAR, SEASON, TIME_BUCKET")
print(f"Columns after adding temporal features: {df.columns.tolist()}")


=== Feature Engineering: Temporal Features ===
Created temporal features: HOUR, DAY, WEEKDAY, IS_WEEKEND, MONTH, YEAR, SEASON, TIME_BUCKET
Columns after adding temporal features: ['BORO_NM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET']


## 2.1. Create Holiday and Payday Features

Create binary flags for holidays and assumed paydays to capture potential temporal patterns related to these events.

In [14]:
print("\n=== Feature Engineering: Holiday and Payday Features ===")
# Recreate a datetime series from YEAR, MONTH, DAY for holiday and payday checking
# Ensure YEAR, MONTH, DAY are present and are not NaN before attempting conversion
if 'YEAR' in df.columns and 'MONTH' in df.columns and 'DAY' in df.columns:
    # Create a temporary date column, handling potential errors by setting to NaT
    # This is crucial if YEAR, MONTH, DAY could form invalid dates (e.g., Feb 30)
    # However, pd.to_datetime will handle standard invalid dates by raising errors if not coerced.
    # For safety, let's ensure components are integer and handle NaNs that might arise from prior steps.
    date_components = df[['YEAR', 'MONTH', 'DAY']].dropna()
    df_dates = pd.to_datetime(date_components, errors='coerce')

    # Initialize US holidays
    us_holidays = holidays.US(years=df_dates.dt.year.unique())

    # Create IS_HOLIDAY column
    # Apply to the original df index to ensure alignment
    df['IS_HOLIDAY'] = 0
    df.loc[df_dates.index, 'IS_HOLIDAY'] = df_dates.dt.date.apply(lambda date: 1 if date in us_holidays else 0).astype(int)
    print(f"Created IS_HOLIDAY feature. {df['IS_HOLIDAY'].sum()} instances on a holiday.")

    # Create IS_PAYDAY column (assuming 1st and 15th of the month)
    df['IS_PAYDAY'] = 0
    df.loc[df_dates.index, 'IS_PAYDAY'] = df_dates.apply(lambda x: 1 if x.day == 1 or x.day == 15 else 0).astype(int)
    print(f"Created IS_PAYDAY feature. {df['IS_PAYDAY'].sum()} instances on an assumed payday.")
else:
    print("Warning: YEAR, MONTH, or DAY column not found. Skipping IS_HOLIDAY and IS_PAYDAY creation.")

print(f"Columns after adding holiday/payday features: {df.columns.tolist()}")


=== Feature Engineering: Holiday and Payday Features ===
Created IS_HOLIDAY feature. 82010 instances on a holiday.
Created IS_PAYDAY feature. 180576 instances on an assumed payday.
Columns after adding holiday/payday features: ['BORO_NM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET', 'IS_HOLIDAY', 'IS_PAYDAY']


## 3. Refine Location Features

Improve location-related features using imputation and rule-based logic:
- Impute missing `PREM_TYP_DESC` based on proximity to POIs (Bar, Nightclub, ATM, Metro) and offense type (`OFNS_DESC`).
- Group `PARKS_NM` entries into `PREM_TYP_DESC` as 'PARK/PLAYGROUND'.
- Impute missing `LOC_OF_OCCUR_DESC` based on `PREM_TYP_DESC` and `OFNS_DESC`.

### 3.1 Impute `PREM_TYP_DESC`

In [15]:
print("\n=== Feature Engineering: Refining Location Features - Impute PREM_TYP_DESC ===")
# 1) Identify rows where PREM_TYP_DESC is missing or placeholder
missing_prem_placeholders = ['(NULL)', None, np.nan, 'UNKNOWN'] # Include UNKNOWN from previous cleaning
mask_null_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders)
initial_missing_prem = mask_null_prem.sum()
print(f"Initial missing/placeholder PREM_TYP_DESC: {initial_missing_prem}")

# 2) Compute nearest POI and its distance (helper columns)
poi_cols = ['BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'METRO_DISTANCE']
df['NEAREST_POI_DIST'] = df[poi_cols].min(axis=1)
df['NEAREST_POI_TYPE'] = df[poi_cols].idxmin(axis=1)

# Mapping from distance column name to PREM_TYP_DESC value
distance_to_prem = {
    'BAR_DISTANCE': 'BAR/NIGHT CLUB',
    'NIGHTCLUB_DISTANCE': 'BAR/NIGHT CLUB',
    'ATM_DISTANCE': 'ATM',
    'METRO_DISTANCE': 'TRANSIT - NYC SUBWAY'
}

# 3) Distance-based imputation: only if nearest POI is within 30 meters
mask_impute_by_dist = mask_null_prem & (df['NEAREST_POI_DIST'] <= 30)
df.loc[mask_impute_by_dist, 'PREM_TYP_DESC'] = df.loc[mask_impute_by_dist, 'NEAREST_POI_TYPE'].map(distance_to_prem)
imputed_by_dist_count = mask_impute_by_dist.sum()
print(f"Imputed {imputed_by_dist_count} PREM_TYP_DESC based on POI proximity (<= 30m).")

# Drop helper columns
df.drop(columns=['NEAREST_POI_DIST', 'NEAREST_POI_TYPE'], inplace=True)

# 4) Fallback using OFNS_DESC for street-related crimes
mask_still_null_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders)
street_crime_keywords = ['BURGLARY', 'ROBBERY', 'ASSAULT', 'GRAND LARCENY', 'PETIT LARCENY', 'UNAUTHORIZED USE OF A VEHICLE']
mask_impute_by_offense = mask_still_null_prem & df['OFNS_DESC'].str.contains('|'.join(street_crime_keywords), na=False, case=False)
df.loc[mask_impute_by_offense, 'PREM_TYP_DESC'] = 'STREET'
imputed_by_offense_count = mask_impute_by_offense.sum()
print(f"Imputed {imputed_by_offense_count} PREM_TYP_DESC as 'STREET' based on OFNS_DESC.")

# 5) Final default for any remaining missing values
mask_final_null_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders)
final_default_count = mask_final_null_prem.sum()
df.loc[mask_final_null_prem, 'PREM_TYP_DESC'] = 'OTHER'
print(f"Filled remaining {final_default_count} missing PREM_TYP_DESC with 'OTHER'.")

# Final check
final_missing_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders).sum()
print(f"PREM_TYP_DESC missing/placeholders after imputation: {final_missing_prem}")


=== Feature Engineering: Refining Location Features - Impute PREM_TYP_DESC ===
Initial missing/placeholder PREM_TYP_DESC: 35590
Imputed 2225 PREM_TYP_DESC based on POI proximity (<= 30m).
Imputed 12816 PREM_TYP_DESC as 'STREET' based on OFNS_DESC.
Filled remaining 20549 missing PREM_TYP_DESC with 'OTHER'.
PREM_TYP_DESC missing/placeholders after imputation: 0


### 3.2 Consolidate Park Information

In [16]:
print("\n=== Feature Engineering: Refining Location Features - Consolidate Park Info ===")
# Group PARKS_NM into PREM_TYP_DESC
if 'PARKS_NM' in df.columns:
    park_mask = df['PARKS_NM'].notna() & (df['PARKS_NM'] != '(NULL)') & (df['PARKS_NM'] != 'UNKNOWN')
    park_update_count = park_mask.sum()
    df.loc[park_mask, 'PREM_TYP_DESC'] = 'PARK/PLAYGROUND'
    print(f"Updated {park_update_count} PREM_TYP_DESC entries to 'PARK/PLAYGROUND' based on PARKS_NM.")
    # Drop the original PARKS_NM column as it's now consolidated
    df.drop(columns=['PARKS_NM'], inplace=True)
    print("Dropped PARKS_NM column.")
else:
    print("PARKS_NM column not found, skipping consolidation.")


=== Feature Engineering: Refining Location Features - Consolidate Park Info ===
Updated 15739 PREM_TYP_DESC entries to 'PARK/PLAYGROUND' based on PARKS_NM.
Dropped PARKS_NM column.


### 3.3 Impute `LOC_OF_OCCUR_DESC`

In [17]:
print("\n=== Feature Engineering: Refining Location Features - Impute LOC_OF_OCCUR_DESC ===")
# 1) Identify missing/placeholder values
missing_loc_placeholders = ['(NULL)', None, np.nan, 'UNKNOWN']
mask_null_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders)
initial_missing_loc = mask_null_loc.sum()
print(f"Initial missing/placeholder LOC_OF_OCCUR_DESC: {initial_missing_loc}")

# 2) Rule-based mapping from PREM_TYP_DESC to LOC_OF_OCCUR_DESC
prem_to_loc = {
    # indoor locations
    'GROCERY/BODEGA':                     'INSIDE',
    'RESIDENCE - APT. HOUSE':             'INSIDE',
    'RESIDENCE-HOUSE':                    'INSIDE',
    'RESIDENCE - PUBLIC HOUSING':         'INSIDE',
    'DEPARTMENT STORE':                   'INSIDE',
    'CHAIN STORE':                        'INSIDE',
    'DRUG STORE':                         'INSIDE',
    'FOOD SUPERMARKET':                   'INSIDE',
    'COMMERCIAL BUILDING':                'INSIDE',
    'BANK':                               'INSIDE',
    'PUBLIC BUILDING':                    'INSIDE',
    'HOTEL/MOTEL':                        'INSIDE',
    'HOMELESS SHELTER':                   'INSIDE',
    'PUBLIC SCHOOL':                      'INSIDE',
    'PRIVATE/PAROCHIAL SCHOOL':           'INSIDE',
    'COLLEGE/UNIVERSITY':                 'INSIDE',
    'HOSPITAL':                           'INSIDE',
    'DOCTOR/DENTIST OFFICE':              'INSIDE',
    'GYM/FITNESS FACILITY':               'INSIDE',
    'BAR/NIGHT CLUB':                     'INSIDE',
    'RESTAURANT/DINER':                   'INSIDE',
    'FAST FOOD':                          'INSIDE',
    'DRY CLEANER/LAUNDRY':                'INSIDE',
    'BEAUTY & NAIL SALON':                'INSIDE',
    'CLOTHING/BOUTIQUE':                  'INSIDE',
    'JEWELRY':                            'INSIDE',
    'PHOTO/COPY':                         'INSIDE',
    'VIDEO STORE':                        'INSIDE',
    'STORE UNCLASSIFIED':                 'INSIDE',
    'SMALL MERCHANT':                     'INSIDE',
    'CANDY STORE':                        'INSIDE',
    'VARIETY STORE':                      'INSIDE',
    'SHOE':                               'INSIDE',
    'CHECK CASHING BUSINESS':             'INSIDE',
    'STORAGE FACILITY':                   'INSIDE',
    'REAL ESTATE':                        'INSIDE',
    'SOCIAL CLUB/POLICY':                 'INSIDE',
    'OTHER HOUSE OF WORSHIP':             'INSIDE',
    'CHURCH':                             'INSIDE',
    'SYNAGOGUE':                          'INSIDE',
    'MOSQUE':                             'INSIDE',
    'DAYCARE FACILITY':                   'INSIDE',
    'ABANDONED BUILDING':                 'INSIDE',
    'LOAN COMPANY':                       'INSIDE',
    'TAXI (YELLOW LICENSED)':             'INSIDE',
    'TAXI (LIVERY LICENSED)':             'INSIDE',
    'TAXI/LIVERY (UNLICENSED)':           'INSIDE',
    'BUS (NYC TRANSIT)':                  'INSIDE',
    'BUS (OTHER)':                        'INSIDE',
    'TRANSIT FACILITY (OTHER)':           'INSIDE',
    'AIRPORT TERMINAL':                   'INSIDE',
    'FERRY/FERRY TERMINAL':               'INSIDE',

    # outdoor or semi‐outdoor locations
    'STREET':                             'REAR',
    'HIGHWAY/PARKWAY':                    'REAR',
    'TUNNEL':                             'REAR',
    'PARK/PLAYGROUND':                    'REAR',
    'OPEN AREAS (OPEN LOTS)':             'REAR',
    'PARKING LOT/GARAGE (PUBLIC)':        'REAR',
    'PARKING LOT/GARAGE (PRIVATE)':       'REAR',
    'GAS STATION':                        'REAR',
    'ATM':                                'REAR',
    'BUS STOP':                           'REAR',
    'BUS TERMINAL':                       'REAR',
    'TRAMWAY':                            'REAR',
    'BRIDGE':                             'REAR',
    'MARINA/PIER':                        'REAR',
    'CEMETERY':                           'REAR',
    'CONSTRUCTION SITE':                  'REAR',
    'MOBILE FOOD':                        'REAR',
    'FERRY/FERRY TERMINAL':               'REAR',  # also treated as terminal interior above

    # catch-all for others
    'OTHER':                              'UNKNOWN',
    'STORE UNCLASSIFIED':                 'UNKNOWN',
    'TRANSIT - NYC SUBWAY':               'UNKNOWN' # Can be inside station, on platform, or on train
}

# Apply the mapping to fill missing LOC_OF_OCCUR_DESC
df.loc[mask_null_loc, 'LOC_OF_OCCUR_DESC'] = df.loc[mask_null_loc, 'PREM_TYP_DESC'].map(prem_to_loc)
imputed_by_prem_count = mask_null_loc.sum() - df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders).sum()
print(f"Imputed {imputed_by_prem_count} LOC_OF_OCCUR_DESC based on PREM_TYP_DESC mapping.")

# 3) Refinement using OFNS_DESC for potential inside crimes
mask_still_null_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders)
inside_crime_keywords = ['BURGLARY', 'TRESPASS', 'ROBBERY', 'ASSAULT'] # Keywords suggesting inside location
mask_impute_by_offense_loc = mask_still_null_loc & df['OFNS_DESC'].str.contains('|'.join(inside_crime_keywords), na=False, case=False)
df.loc[mask_impute_by_offense_loc, 'LOC_OF_OCCUR_DESC'] = 'INSIDE'
imputed_by_offense_loc_count = mask_impute_by_offense_loc.sum()
print(f"Imputed {imputed_by_offense_loc_count} LOC_OF_OCCUR_DESC as 'INSIDE' based on OFNS_DESC.")

# 4) Final default for any remaining missing values
mask_final_null_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders)
final_default_loc_count = mask_final_null_loc.sum()
df.loc[mask_final_null_loc, 'LOC_OF_OCCUR_DESC'] = 'UNKNOWN'
print(f"Filled remaining {final_default_loc_count} missing LOC_OF_OCCUR_DESC with 'UNKNOWN'.")

# Final check
final_missing_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders).sum()
print(f"LOC_OF_OCCUR_DESC missing/placeholders after imputation: {final_missing_loc}")


=== Feature Engineering: Refining Location Features - Impute LOC_OF_OCCUR_DESC ===
Initial missing/placeholder LOC_OF_OCCUR_DESC: 464374
Imputed 378151 LOC_OF_OCCUR_DESC based on PREM_TYP_DESC mapping.
Imputed 15240 LOC_OF_OCCUR_DESC as 'INSIDE' based on OFNS_DESC.
Filled remaining 70983 missing LOC_OF_OCCUR_DESC with 'UNKNOWN'.
LOC_OF_OCCUR_DESC missing/placeholders after imputation: 70983


## 5. Create Demographic Interaction Features

Generate features indicating whether the suspect and victim share the same demographic characteristics:
- `SAME_AGE_GROUP`: 1 if suspect and victim age groups match, 0 otherwise.
- `SAME_SEX`: 1 if suspect and victim sexes match, 0 otherwise.

In [18]:
print("\n=== Feature Engineering: Demographic Interaction Features ===")
# 1) SAME_AGE_GROUP: 1 if suspect and victim share the same age group, else 0
df['SAME_AGE_GROUP'] = 0
# Define valid (non-null, non-placeholder) age-group rows
valid_age_placeholders = ['UNKNOWN', '(NULL)', None, np.nan]
age_valid = (~df['SUSP_AGE_GROUP'].isin(valid_age_placeholders)) & (~df['VIC_AGE_GROUP'].isin(valid_age_placeholders))
# Set flag where age groups match
df.loc[age_valid & (df['SUSP_AGE_GROUP'] == df['VIC_AGE_GROUP']), 'SAME_AGE_GROUP'] = 1
print(f"Created SAME_AGE_GROUP feature. {df['SAME_AGE_GROUP'].sum()} instances where age groups match.")

# 2) SAME_SEX: 1 if suspect and victim have the same sex, else 0
df['SAME_SEX'] = 0
# Define valid (non-null, non-placeholder) sex rows
valid_sex_placeholders = ['U', 'UNKNOWN', '(NULL)', None, np.nan] # 'U' often means Unknown
sex_valid = (~df['SUSP_SEX'].isin(valid_sex_placeholders)) & (~df['VIC_SEX'].isin(valid_sex_placeholders))
# Set flag where sexes match
df.loc[sex_valid & (df['SUSP_SEX'] == df['VIC_SEX']), 'SAME_SEX'] = 1
print(f"Created SAME_SEX feature. {df['SAME_SEX'].sum()} instances where sexes match.")


=== Feature Engineering: Demographic Interaction Features ===
Created SAME_AGE_GROUP feature. 388596 instances where age groups match.
Created SAME_SEX feature. 515461 instances where sexes match.


In [19]:
print("\n=== Feature Engineering: TO_CHECK_CITIZENS Feature ===")
# List of offense descriptions that trigger the flag
to_check_citizens_list = [
    'CRIMINAL TRESPASS', 'CRIMINAL MISCHIEF & RELATED OF', 'HARRASSMENT 2',
    'GRAND LARCENY OF MOTOR VEHICLE', 'PETIT LARCENY', 'GRAND LARCENY',
    'INTOXICATED & IMPAIRED DRIVING', 'FRAUDS', 'THEFT-FRAUD',
    'OFF. AGNST PUB ORD SENSBLTY &', 'BURGLARY', 'ROBBERY', 'FELONY ASSAULT',
    'ASSAULT 3 & RELATED OFFENSES', 'DANGEROUS DRUGS', 'RAPE', 'SEX CRIMES',
    'DANGEROUS WEAPONS', 'ARSON', 'POSSESSION OF STOLEN PROPERTY',
    'UNAUTHORIZED USE OF A VEHICLE', 'FORGERY', 'ENDAN WELFARE INCOMP',
    'OTHER OFFENSES RELATED TO THEF', 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
    'KIDNAPPING & RELATED OFFENSES', 'OTHER OFFENSES RELATED TO THEFT', # Duplicate, keep one
    'OFFENSES RELATED TO CHILDREN', 'BURGLAR\S TOOLS', 'ESCAPE 3',
    'CANNABIS RELATED OFFENSES', 'PETIT LARCENY OF MOTOR VEHICLE',
    'FELONY SEX CRIMES', 'PROSTITUTION & RELATED OFFENSES', 'JOSTLING',
    'KIDNAPPING', 'DISORDERLY CONDUCT', 'INTOXICATED/IMPAIRED DRIVING',
    'DISRUPTION OF A RELIGIOUS SERV', 'FRAUDULENT ACCOSTING', 'THEFT OF SERVICES',
    'UNRSNBLE NOISE', 'OTHER TRAFFIC INFRACTION', 'LOITERING/GAMBLING (CARDS, DIC',
    'UNLAWFUL POSS. WEAP. ON SCHOOL', 'FORTUNE TELLING', 'LOITERING',
    'FAIL RPT WOUNDS'
]
# Remove duplicates from the list
to_check_citizens_list = list(dict.fromkeys(to_check_citizens_list))

# Create the boolean column based on the condition
df['TO_CHECK_CITIZENS'] = df['OFNS_DESC'].isin(to_check_citizens_list).astype(int)
print(f"Created TO_CHECK_CITIZENS feature. {df['TO_CHECK_CITIZENS'].sum()} instances flagged.")


=== Feature Engineering: TO_CHECK_CITIZENS Feature ===
Created TO_CHECK_CITIZENS feature. 2270901 instances flagged.


In [20]:
print("\n=== Dropping Rows with NaN values ===")
initial_rows = len(df)
print(f"Initial number of rows: {initial_rows}")

# Drop rows that contain any NaN value
df_cleaned = df.dropna()

final_rows = len(df_cleaned)
rows_dropped = initial_rows - final_rows

print(f"Number of rows after dropping NaNs: {final_rows}")
print(f"Number of rows dropped: {rows_dropped}")

# Update the main DataFrame reference to the cleaned one
df = df_cleaned

print(f"Current DataFrame shape: {df.shape}")


=== Dropping Rows with NaN values ===
Initial number of rows: 2496759
Number of rows after dropping NaNs: 2496742
Number of rows dropped: 17
Current DataFrame shape: (2496742, 44)


In [21]:
print("\n=== Saving Feature Engineered Dataset (Before Scaling/PCA) ===")
# Ensure feature_engineering_dir and feature_engineered_file_path are defined

if 'df' in globals() and 'feature_engineered_file_path' in globals() and 'os' in globals():
    try:
        df.to_csv(feature_engineered_file_path, index=False)
        print(f"Feature engineered dataset (before scaling/PCA) saved successfully to: {feature_engineered_file_path}")
        print(f"Dataset shape: {df.shape}")
        print(f"Columns in the dataset: {df.columns.tolist()}")
    except NameError as ne:
        print(f"A NameError occurred while trying to save: {ne}. Ensure 'df' and path variables are correctly defined.")
    except Exception as e:
        print(f"Error saving feature engineered dataset (before scaling/PCA): {e}")
else:
    print("Error: 'df' or 'feature_engineered_file_path' or 'os' not found in globals. Cannot save the intermediate DataFrame.")


=== Saving Feature Engineered Dataset (Before Scaling/PCA) ===
Feature engineered dataset (before scaling/PCA) saved successfully to: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\FeatureEngineered\feature_engineered_crime_data.csv
Dataset shape: (2496742, 44)
Columns in the dataset: ['BORO_NM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET', 'IS_HOLIDAY', 'IS_PAYDAY', 'SAME_AGE_GROUP', 'SAME_SEX', 'TO_CHECK_CITIZENS']


## Attribute Type Classification

Below is a classification of each column in the final feature-engineered dataset according to its data type and semantic meaning, *before* encoding and transformation. This helps guide the next steps in the Data Reduction and Transformation phase.

| Column                | Data Type                     | Details/Notes                                  | Transformation Needed | Encoding Type (if needed) |
|-----------------------|-------------------------------|------------------------------------------------|-----------------------|---------------------------|
| BORO_NM               | Categorical (nominal)         | Borough name                                   | Encoding              | One-Hot               |         |
| KY_CD            | Categorical (nominal)         | Crime classification code | Encoding              | One-Hot                   |       |
| LAW_CAT_CD            | Categorical (ordinal)         | Crime level (FELONY > MISDEMEANOR > VIOLATION) | Encoding              | Ordinal                   |       |
| LOC_OF_OCCUR_DESC     | Categorical (nominal)         | Location description (INSIDE, FRONT, etc.)     | Encoding              | One-Hot                   |
| OFNS_DESC             | Categorical (nominal)         | Offense description                            | Encoding              | One-Hot (High Cardinality)|
| PD_CD                 | Categorical (nominal)         | Detailed police code                           | Encoding              | One-Hot (High Cardinality)|
| PREM_TYP_DESC         | Categorical (nominal)         | Premises type description                      | Encoding              | One-Hot (High Cardinality)|
| SUSP_AGE_GROUP        | Categorical (ordinal)         | Age group (e.g., '<18', '18-24', 'UNKNOWN')    | Encoding              | Ordinal                   |
| SUSP_RACE             | Categorical (nominal)         | Suspect race                                   | Encoding              | One-Hot                   |
| SUSP_SEX              | Categorical (nominal)         | Suspect sex (M/F/U)                            | Encoding              | One-Hot                   |
| VIC_AGE_GROUP         | Categorical (ordinal)         | Age group (e.g., '<18', '18-24', 'UNKNOWN')    | Encoding              | Ordinal                   |
| VIC_RACE              | Categorical (nominal)         | Victim race                                    | Encoding              | One-Hot                   |
| VIC_SEX               | Categorical (nominal)         | Victim sex (M/F/D/E/U)                         | Encoding              | One-Hot                   |
| LATITUDE              | Numeric (continuous)          | Geographic coordinate                          | Scaling               | -                         |
| LONGITUDE             | Numeric (continuous)          | Geographic coordinate                          | Scaling               | -                         |
| BAR_DISTANCE          | Numeric (continuous)          | Distance to nearest bar (meters)               | Scaling               | -                         |
| NIGHTCLUB_DISTANCE    | Numeric (continuous)          | Distance to nearest nightclub (meters)         | Scaling               | -                         |
| ATM_DISTANCE          | Numeric (continuous)          | Distance to nearest ATM (meters)               | Scaling               | -                         |
| METRO_DISTANCE        | Numeric (continuous)          | Distance to nearest metro station (meters)     | Scaling               | -                         |
| ATMS_COUNT            | Numeric (discrete)            | Count of ATMs nearby                           | Scaling               | -                         |
| BARS_COUNT            | Numeric (discrete)            | Count of bars nearby                           | Scaling               | -                         |
| BUS_STOPS_COUNT       | Numeric (discrete)            | Count of bus stops nearby                      | Scaling               | -                         |
| METROS_COUNT | Numeric (discrete)            | Count of metro stations nearby                 | Scaling               | -                         |
| NIGHTCLUBS_COUNT      | Numeric (discrete)            | Count of nightclubs nearby                     | Scaling               | -                         |
| SCHOOLS_COUNT         | Numeric (discrete)            | Count of schools nearby                        | Scaling               | -                         |
| HOUR                  | Numeric (cyclical)            | Hour of the day (0-23)                         | Encoding              | Cyclical (Sine/Cosine)    |
| DAY                   | Numeric (cyclical)            | Day of the month (1-31)                        | Encoding              | Cyclical (Sine/Cosine)    |
| WEEKDAY               | Categorical (cyclical)        | Day of the week (MON-SUN)                      | Encoding              | Cyclical (Sine/Cosine)    |
| IS_WEEKEND            | Binary                        | 0 = weekday, 1 = weekend                       | None                  | -                         |
| MONTH                 | Numeric (cyclical)            | Month of the year (1-12)                       | Encoding              | Cyclical (Sine/Cosine)    |
| SEASON                | Categorical (nominal)         | Season (WINTER, SPRING, SUMMER, AUTUMN)        | Encoding              | One-Hot                   |
| TIME_BUCKET           | Categorical (nominal)         | Time of day (NIGHT, MORNING, AFTERNOON, EVENING) | Encoding              | One-Hot                   |
| YEAR                  | Numeric (discrete)            | Year of the event                              | Scaling               | -                         |
| IS_HOLIDAY            | Binary                        | 0 = not a holiday, 1 = holiday                 | None                  | -                         |
| IS_PAYDAY             | Binary                        | 0 = not a payday, 1 = payday (1st or 15th)     | None                  | -                         |
| SAME_AGE_GROUP        | Binary                        | 1 if suspect/victim age groups match, else 0   | None                  | -                         |
| SAME_SEX              | Binary                        | 1 if suspect/victim sexes match, else 0        | None                  | -                         |
| TO_CHECK_CITIZENS     | Binary                        | 1 if offense is relevant for citizens, else 0 | None                  | -                         |
| TOTAL_POI_COUNT       | Numeric (discrete)            | Sum of all POI counts  | Scaling               | -                         |
| POI_DIVERSITY         | Numeric (discrete)            | Number of different POI types present          | Scaling               | -                         |
| POI_DENSITY_SCORE      | Numeric (continuous)           | Density score                           | Scaling                  | -                             |
| MIN_POI_DISTANCE      | Numeric (continuous)          | Distance to closest POI of any type            | Scaling               | -                         |
| AVG_POI_DISTANCE      | Numeric (continuous)          | Average distance to POIs                       | Scaling               | -                         |
| MAX_POI_DISTANCE       | Numeric (continuous)           | Maximum distance to POIs                       | Scaling                  | -                             |

### Reasons for Cyclical/Ordinal Encoding

Cyclical and Ordinal encoding are valuable techniques for representing categorical features while preserving inherent relationships and limiting dimensionality increase compared to One-Hot encoding.
- **Cyclical Encoding:** This is particularly useful for features where the end of the sequence connects back to the beginning, such as time-related attributes (`HOUR`, `DAY`, `WEEKDAY`, `MONTH`). For example, hour 23 is conceptually close to hour 0, December is close to January. Cyclical encoding (often using sine and cosine transformations) captures this wrap-around nature, preventing the model from incorrectly interpreting these values as distant.
- **Ordinal Encoding:** This method is suitable for features with a clear, inherent order or ranking (`LAW_CAT_CD`, `SUSP_AGE_GROUP`, `VIC_AGE_GROUP`). Assigning sequential integers (e.g., 0, 1, 2...) allows the model to understand the relative magnitude or progression between categories (e.g., 'FELONY' > 'MISDEMEANOR', '<18' < '18-24').

### Reasons for One-Hot Encoding

One-Hot encoding is applied to nominal categorical features where there is no inherent order or ranking among the categories (`BORO_NM`, `KY_CD`, `LOC_OF_OCCUR_DESC`, `OFNS_DESC`, `PD_CD`, `PREM_TYP_DESC`, `SUSP_RACE`, `SUSP_SEX`, `VIC_RACE`, `VIC_SEX`, `SEASON`, `TIME_BUCKET`). It creates new binary (0 or 1) columns for each unique category in the original feature. This prevents the model from assuming any ordinal relationship between categories that doesn't exist. While it can significantly increase the number of features (especially for high-cardinality columns like `OFNS_DESC`), it ensures that each category is treated independently.

### Reasons for Scaling

Scaling is essential for numeric features (`Latitude`, `Longitude`, distance columns, count columns, `YEAR`) that have different ranges or units. Many machine learning algorithms (especially those based on distance calculations like k-NN or SVM, or those using gradient descent like linear regression and neural networks) are sensitive to the scale of input features. Features with larger values might disproportionately influence the model's learning process. Scaling methods like Standardization (Z-score normalization) or Min-Max scaling transform the data to a common scale (e.g., mean 0 and standard deviation 1, or range [0, 1]), ensuring that all features contribute more equally to the model's outcome.

## Ordinal Encoding

In [22]:
print("\n=== Feature Engineering: Ordinal Encoding ===")

# Ordinal categories for correct severity order
law_cat_order = ['VIOLATION', 'MISDEMEANOR', 'FELONY']
age_group_order = ['<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN']

enc = OrdinalEncoder(categories=[law_cat_order, age_group_order, age_group_order])
cols_to_encode = ['LAW_CAT_CD', 'SUSP_AGE_GROUP', 'VIC_AGE_GROUP']
df[cols_to_encode] = enc.fit_transform(df[cols_to_encode]).astype(int)

for feature, cats in zip(cols_to_encode, enc.categories_):
    print(f"{feature} categories → {list(cats)}")

print(f"Ordinal encoding applied to: {cols_to_encode}")


=== Feature Engineering: Ordinal Encoding ===
LAW_CAT_CD categories → ['VIOLATION', 'MISDEMEANOR', 'FELONY']
SUSP_AGE_GROUP categories → ['<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN']
VIC_AGE_GROUP categories → ['<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN']
Ordinal encoding applied to: ['LAW_CAT_CD', 'SUSP_AGE_GROUP', 'VIC_AGE_GROUP']


## Cyclical Encoding

In [23]:
print("\n=== Feature Engineering: Cyclical Encoding ===")

weekday_order    = ['MONDAY','TUESDAY','WEDNESDAY','THURSDAY','FRIDAY','SATURDAY','SUNDAY']

# Create WEEKDAY_NUM if WEEKDAY column exists
if 'WEEKDAY' in df.columns:
    df['WEEKDAY_NUM'] = df['WEEKDAY'].map({d:i for i,d in enumerate(weekday_order)})
    print("Created 'WEEKDAY_NUM' from 'WEEKDAY'.")
else:
    print("Warning: 'WEEKDAY' column not found, cannot create 'WEEKDAY_NUM'.")

# Apply cyclical encoding if source columns exist
encoded_features = []
original_cyclical_cols_to_drop = []

if 'HOUR' in df.columns:
    df['HOUR_SIN']         = np.sin(2*np.pi*df['HOUR']/24)
    df['HOUR_COS']         = np.cos(2*np.pi*df['HOUR']/24)
    encoded_features.append('HOUR')
    original_cyclical_cols_to_drop.append('HOUR')
else:
    print("Warning: 'HOUR' column not found, skipping its cyclical encoding.")

if 'DAY' in df.columns:
    df['DAY_SIN']          = np.sin(2*np.pi*df['DAY']/31) # Using 31 as max days in a month
    df['DAY_COS']          = np.cos(2*np.pi*df['DAY']/31)
    encoded_features.append('DAY')
    original_cyclical_cols_to_drop.append('DAY')
else:
    print("Warning: 'DAY' column not found, skipping its cyclical encoding.")

if 'WEEKDAY_NUM' in df.columns: # Relies on WEEKDAY_NUM
    df['WEEKDAY_SIN']      = np.sin(2*np.pi*df['WEEKDAY_NUM']/7)
    df['WEEKDAY_COS']      = np.cos(2*np.pi*df['WEEKDAY_NUM']/7)
    encoded_features.append('WEEKDAY')
    if 'WEEKDAY' in df.columns: # Ensure original WEEKDAY (string) is also dropped
        original_cyclical_cols_to_drop.append('WEEKDAY')
else:
    print("Warning: 'WEEKDAY_NUM' column not found, skipping WEEKDAY cyclical encoding.")

if 'MONTH' in df.columns:
    df['MONTH_SIN']        = np.sin(2*np.pi*df['MONTH']/12)
    df['MONTH_COS']        = np.cos(2*np.pi*df['MONTH']/12)
    encoded_features.append('MONTH')
    original_cyclical_cols_to_drop.append('MONTH')
else:
    print("Warning: 'MONTH' column not found, skipping its cyclical encoding.")

# Drop temporary and original cyclical columns that were created and exist
columns_to_drop_intermediate = ['WEEKDAY_NUM']
existing_intermediate_to_drop = [col for col in columns_to_drop_intermediate if col in df.columns]
if existing_intermediate_to_drop:
    df.drop(columns=existing_intermediate_to_drop, inplace=True)
    print(f"Dropped temporary intermediate columns: {existing_intermediate_to_drop}")

existing_original_cyclical_to_drop = [col for col in original_cyclical_cols_to_drop if col in df.columns]
if existing_original_cyclical_to_drop:
    df.drop(columns=existing_original_cyclical_to_drop, inplace=True)
    print(f"Dropped original cyclical source columns: {existing_original_cyclical_to_drop}")

if encoded_features:
    print(f"Cyclical encoding (sin/cos) applied to: {', '.join(encoded_features)}.")
else:
    print("No cyclical encoding was applied as relevant columns were not found.")


=== Feature Engineering: Cyclical Encoding ===
Created 'WEEKDAY_NUM' from 'WEEKDAY'.
Dropped temporary intermediate columns: ['WEEKDAY_NUM']
Dropped original cyclical source columns: ['HOUR', 'DAY', 'WEEKDAY', 'MONTH']
Cyclical encoding (sin/cos) applied to: HOUR, DAY, WEEKDAY, MONTH.


## One Hot Encoding

One-Hot encoding is used for nominal categorical features where no ordinal relationship exists. It creates a new binary column for each category, indicating its presence (1) or absence (0). This method is suitable for columns like `BORO_NM`, `KY_CD`, `LOC_OF_OCCUR_DESC`, `OFNS_DESC`, `PD_CD`, `PREM_TYP_DESC`, `SUSP_RACE`, `SUSP_SEX`, `VIC_RACE`, `VIC_SEX`, `SEASON`, and `TIME_BUCKET`. While it can increase dimensionality, especially with high-cardinality features, it ensures the model does not infer an incorrect order between categories.



In [24]:
print("\n=== Feature Engineering: One Hot Encoding ===")

# Features to one-hot encode (nominal categorical, not ordinal/cyclical)
# These are columns that do not have an inherent order and are not cyclical.
cols_to_encode = [
    'BORO_NM', 'KY_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PD_CD',
    'PREM_TYP_DESC', 'SUSP_RACE', 'SUSP_SEX', 'VIC_RACE', 'VIC_SEX',
    'SEASON', 'TIME_BUCKET'
]

# Filter out columns that might not exist in the DataFrame to avoid errors
existing_cols_to_encode = [col for col in cols_to_encode if col in df.columns]
if existing_cols_to_encode:
    # Apply one-hot encoding using pandas get_dummies
    # drop_first=False is used to avoid issues with interpretability or specific model requirements.
    df = pd.get_dummies(df, columns=existing_cols_to_encode, prefix=existing_cols_to_encode, dtype=np.uint8)
    print("Columns converted with One-Hot Encoding:", existing_cols_to_encode)
    print(f"Final shape after encoding: {df.shape}")
else:
    print("No specified columns for One-Hot Encoding found in the DataFrame.")

# Display a sample of the DataFrame to check new columns (optional)
# print("\nSample of DataFrame after One-Hot Encoding:")
# print(df.sample(5))


=== Feature Engineering: One Hot Encoding ===
Columns converted with One-Hot Encoding: ['BORO_NM', 'KY_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_RACE', 'SUSP_SEX', 'VIC_RACE', 'VIC_SEX', 'SEASON', 'TIME_BUCKET']
Final shape after encoding: (2496742, 699)


## Data Scaling and Dimensionality Reduction (PCA)

In this section, we apply different scaling techniques—**StandardScaler**, **MinMaxScaler**, and **RobustScaler**—to all numeric features of the dataset. The goal is to compare these normalization methods and identify which one is most suitable for our data, especially in preparation for dimensionality reduction.

After scaling, we perform **Principal Component Analysis (PCA)** on each version of the encoded dataset (i.e., after all categorical variables have been encoded). PCA is used to reduce the number of features while retaining 95% of the total variance, making the dataset more manageable for subsequent modeling.

By visualizing the cumulative explained variance and the distributions of selected features, we can objectively choose the best scaling method to use for the final, definitive PCA and downstream machine learning tasks.

In [25]:
import gc
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
#import matplotlib.pyplot as plt # Ensure plt is imported if plots are enabled

# Define the target column
target_column = 'LAW_CAT_CD'
# The global 'df' DataFrame is assumed to be the one after all previous feature engineering and encoding steps.

print("\n=== Data Scaling and Dimensionality Reduction (PCA) ===")

# 1. Separate Target Variable and Features
if target_column in df.columns:
    y_target = df[target_column].copy()
    X_features = df.drop(columns=[target_column])
    print(f"Target column '{target_column}' separated. X_features shape: {X_features.shape}, y_target shape: {y_target.shape}")
else:
    print(f"Warning: Target column '{target_column}' not found in DataFrame. PCA will proceed on all numeric columns of the current 'df' (which will be treated as X_features).")
    y_target = None
    X_features = df.copy() # Treat the whole df as features for PCA processing

# 2. Identify Feature Types in X_features
numeric_cols_in_X = X_features.select_dtypes(include=[np.number]).columns.tolist()
# Preserve non-numeric columns from X_features, ensuring their index is maintained for later concatenation
non_numeric_cols_from_X = X_features.select_dtypes(exclude=[np.number])

# 3. Handle No Numeric Features Case
if not numeric_cols_in_X:
    print("Error: No numeric features found in X_features for PCA. Skipping scaling and PCA.")
    # Reconstruct df with original X_features (which includes non-numeric) and y_target if it exists
    # The global 'df' will be updated to reflect this state.
    if y_target is not None:
        # Ensure original df index is preserved if possible, or reset if combining disparate parts
        df = pd.concat([X_features.reset_index(drop=True), y_target.to_frame(name=target_column).reset_index(drop=True)], axis=1)
    else:
        df = X_features # df is already X_features if y_target was None
    print(f"Global 'df' (no PCA performed) shape: {df.shape}")
else:
    print(f"Numeric features for PCA ({len(numeric_cols_in_X)}): {numeric_cols_in_X[:5]}...")
    if not non_numeric_cols_from_X.empty:
        print(f"Non-numeric features to carry over ({len(non_numeric_cols_from_X.columns)}): {non_numeric_cols_from_X.columns.tolist()[:5]}...")

    # 4. Scaling and PCA (on numeric features of X_features)
    X_numeric_features_to_scale = X_features[numeric_cols_in_X]

    # Sample for scaler selection
    sample_size = min(50000, len(X_numeric_features_to_scale))
    X_numeric_sample = X_numeric_features_to_scale.sample(n=sample_size, random_state=42)

    scaler_std = StandardScaler()
    X_sample_std_scaled = scaler_std.fit_transform(X_numeric_sample)
    pca_std_sample = PCA(n_components=0.95, random_state=42).fit(X_sample_std_scaled)
    n_std = pca_std_sample.n_components_

    scaler_mm = MinMaxScaler()
    X_sample_mm_scaled = scaler_mm.fit_transform(X_numeric_sample)
    pca_mm_sample = PCA(n_components=0.95, random_state=42).fit(X_sample_mm_scaled)
    n_mm = pca_mm_sample.n_components_

    scaler_rb = RobustScaler()
    X_sample_rb_scaled = scaler_rb.fit_transform(X_numeric_sample)
    pca_rb_sample = PCA(n_components=0.95, random_state=42).fit(X_sample_rb_scaled)
    n_rb = pca_rb_sample.n_components_

    # Plotting code is commented out as per original
    # import matplotlib.pyplot as plt
    # plt.figure(figsize=(10,6))
    # plt.plot(np.cumsum(pca_std_sample.explained_variance_ratio_), label='StandardScaler (Z-score)')
    # plt.plot(np.cumsum(pca_mm_sample.explained_variance_ratio_), label='MinMaxScaler')
    # plt.plot(np.cumsum(pca_rb_sample.explained_variance_ratio_), label='RobustScaler')
    # plt.xlabel('Number of components')
    # plt.ylabel('Cumulative explained variance')
    # plt.title('PCA: Cumulative Explained Variance by Normalization Method (Sample)')
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    print(f"StandardScaler (sample): {n_std} components for 95% variance")
    print(f"MinMaxScaler (sample):   {n_mm} components for 95% variance")
    print(f"RobustScaler (sample):   {n_rb} components for 95% variance")

    del X_numeric_sample, X_sample_std_scaled, X_sample_mm_scaled, X_sample_rb_scaled
    del pca_std_sample, pca_mm_sample, pca_rb_sample
    gc.collect()

    chosen_scaler = None
    best_scaler_name = ""
    if n_std <= n_mm and n_std <= n_rb:
        best_scaler_name = "StandardScaler"
        chosen_scaler = StandardScaler()
    elif n_mm <= n_std and n_mm <= n_rb:
        best_scaler_name = "MinMaxScaler"
        chosen_scaler = MinMaxScaler()
    else:
        best_scaler_name = "RobustScaler"
        chosen_scaler = RobustScaler()

    print(f"\nUsing {best_scaler_name} for full dataset scaling on numeric features of X_features.")
    # Apply scaling to the full set of numeric features from X_features
    X_scaled_full_numeric_features = chosen_scaler.fit_transform(X_numeric_features_to_scale)

    pca_final = PCA(n_components=0.95, random_state=42)
    # Apply PCA to the scaled numeric features
    X_pca_transformed_numeric_features = pca_final.fit_transform(X_scaled_full_numeric_features)
    num_pca_components = X_pca_transformed_numeric_features.shape[1]
    print(f"Full PCA with {best_scaler_name}: {num_pca_components} components for 95% variance (on numeric part of X_features)")

    # Create DataFrame for PCA components, maintaining original index from X_features
    df_pca_components = pd.DataFrame(X_pca_transformed_numeric_features,
                                     columns=[f'PC{i+1}' for i in range(num_pca_components)],
                                     index=X_features.index) # Use index from X_features

    # 5. Reconstruct the global 'df'
    # Start with PCA components (already indexed like X_features)
    final_df_parts = [df_pca_components]

    # Add non-numeric columns (which were already indexed correctly from X_features)
    if not non_numeric_cols_from_X.empty:
        final_df_parts.append(non_numeric_cols_from_X) # non_numeric_cols_from_X also has index from X_features

    # Concatenate PCA components and non-numeric features
    df_transformed_features = pd.concat(final_df_parts, axis=1)

    # Add back the target variable if it was separated, ensuring index alignment
    if y_target is not None:
        # y_target has the same index as original df, and thus X_features
        df = pd.concat([df_transformed_features, y_target.to_frame(name=target_column)], axis=1)
    else:
        df = df_transformed_features # If no target, df is just the transformed features

    print(f"Global 'df' updated. Final shape: {df.shape}")
    print(f"Final columns (first 20): {df.columns.tolist()[:20]}...")

# The global 'df' is now the DataFrame with feature engineering, encoding, scaling (on numeric features),
# and PCA (on scaled numeric features), and includes the target variable if present.
# It is ready to be saved by a subsequent cell.
# The original df_pca_full logic is now integrated into updating the global 'df'.


=== Data Scaling and Dimensionality Reduction (PCA) ===
Target column 'LAW_CAT_CD' separated. X_features shape: (2496742, 698), y_target shape: (2496742,)
Numeric features for PCA (698): ['SUSP_AGE_GROUP', 'VIC_AGE_GROUP', 'Latitude', 'Longitude', 'BAR_DISTANCE']...
StandardScaler (sample): 384 components for 95% variance
MinMaxScaler (sample):   90 components for 95% variance
RobustScaler (sample):   66 components for 95% variance

Using RobustScaler for full dataset scaling on numeric features of X_features.


MemoryError: Unable to allocate 13.0 GiB for an array with shape (698, 2496742) and data type float64

In [ ]:
print("\n=== Saving Final Processed Dataset ===")
# The global 'df' now contains the data after feature engineering, encoding, scaling, and PCA (if applicable),
# with the target variable re-attached if it was separated.

# Define the path for the final dataset using feature_engineering_dir (defined in an earlier cell)
# Ensure 'os' module is imported if not already (it was imported in cell 3 in the provided notebook content)
# import os
final_processed_file_name = "feature_engineered_scaled_pca_crime_data.csv"
# Ensure feature_engineering_dir is defined. It was defined in cell 60e02e84 as:
# feature_engineering_dir = os.path.join(base_dir, "FeatureEngineered")
# base_dir = "/drive/MyDrive/Data Mining and Machine Learning/Progetto"
# So, final_processed_file_path will be relative to that Google Drive path if run in Colab,
# or a local path if base_dir is defined differently in the execution environment.

if 'feature_engineering_dir' not in globals() and 'os' not in globals():
    import os
    # Attempt to define feature_engineering_dir based on common notebook structure if not found
    # This is a fallback and might need adjustment based on actual execution environment
    print("Warning: 'feature_engineering_dir' or 'os' not found in globals. Attempting to define os and a default path.")
    if 'base_dir' in globals():
         feature_engineering_dir = os.path.join(globals()['base_dir'], "FeatureEngineered")
         os.makedirs(feature_engineering_dir, exist_ok=True)
    else:
        # Fallback to a local directory if base_dir is also not found
        feature_engineering_dir = "FeatureEngineered"
        os.makedirs(feature_engineering_dir, exist_ok=True)
        print(f"Warning: 'base_dir' not found. Using local directory: {feature_engineering_dir}")
elif 'os' not in globals():
    import os
    print("Warning: 'os' module not found in globals. Imported os.")

final_processed_file_path = os.path.join(feature_engineering_dir, final_processed_file_name)

try:
    df.to_csv(final_processed_file_path, index=False)
    print(f"Final processed dataset saved successfully to: {final_processed_file_path}")
    print(f"Final dataset shape: {df.shape}")
except NameError as ne:
    if 'df' in str(ne):
        print("Error: 'df' is not defined. Cannot save the DataFrame.")
    elif 'feature_engineering_dir' in str(ne):
        print("Error: 'feature_engineering_dir' is not defined. Please ensure it's defined in an earlier cell or manually define it.")
        print(f"Attempted to use path: {final_processed_file_path}")
    else:
        print(f"A NameError occurred: {ne}")
except Exception as e:
    print(f"Error saving final processed dataset: {e}")

# Display a sample of the final DataFrame for verification
print("\nSample of the final saved DataFrame (first 5 rows):")
try:
    print(df.head())
except NameError:
    print("Error: 'df' is not defined. Cannot display head.")


# ML Pipeline Summary and Data Quality Report

Comprehensive summary of the feature engineering process and ML pipeline implementation including data validation and reproducibility measures.

In [ ]:
print("\n=== Final Data Scan (Unique Values) ===")
print(f"Final Shape: {df.shape}")
print(f"Final Columns: {df.columns.tolist()}")

# Calculate cardinality for each column
cardinality_info = []
for column in df.columns:
    unique_values = df[column].unique()
    num_unique = len(unique_values)
    cardinality_info.append((column, num_unique, unique_values))

    # Print individual column details (as before)
    print(f"\n###### Column: {column} ######")
    print(f"Number of unique values: {num_unique}")
    limit = 20
    if num_unique > limit:
        print(f"Sample unique values (first {limit}): {unique_values[:limit]}")
    else:
        print(f"Unique values: {unique_values}")

# Sort by cardinality (number of unique values) - descending
cardinality_info.sort(key=lambda x: x[1], reverse=True) # Corrected lambda

print("\n=== Top 10 Columns by Cardinality ===")
for col, count, _ in cardinality_info[:10]: # Adjusted to unpack correctly
    print(f"  {col}: {count} unique values")

print("\n=== ML PIPELINE SUMMARY AND DATA QUALITY REPORT ===")

# Dataset Statistics
print(f"\n--- Dataset Statistics ---")
print(f"Final Dataset Shape: {df.shape}")
print(f"Total Features: {df.shape[1]}")

# Data Quality Metrics
print(f"\n--- Data Quality Metrics ---")
total_missing = df.isnull().sum().sum()
total_cells = df.shape[0] * df.shape[1]
missing_percentage = (total_missing / total_cells) * 100

print(f"Total Missing Values: {total_missing}")
print(f"Missing Data Percentage: {missing_percentage:.2f}%")
print(f"Complete Rows: {df.dropna().shape[0]} ({df.dropna().shape[0]/len(df)*100:.1f}%)")

# Feature Type Distribution
print(f"\n--- Feature Type Distribution ---")
feature_types = df.dtypes.value_counts()
for dtype, count in feature_types.items():
    print(f"  {dtype}: {count} columns")

# Memory Usage
print(f"\n--- Memory Usage ---")
memory_usage = df.memory_usage(deep=True).sum() / 1024**2
print(f"Dataset Memory Usage: {memory_usage:.2f} MB")

# Categorical Feature Cardinality
print(f"\n--- Categorical Feature Cardinality (Top 10) ---")
cat_cols = df.select_dtypes(include=['object']).columns
cardinality_info = []
for col in cat_cols:
    unique_count = df[col].nunique()
    cardinality_info.append((col, unique_count))

cardinality_info.sort(key=lambda x: x[1], reverse=True)
for col, count in cardinality_info[:10]:
    print(f"  {col}: {count} unique values")

# Temporal Feature Summary
print(f"\n--- Temporal Features Created ---")
temporal_features = ['HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET', 'IS_HOLIDAY', 'IS_PAYDAY']
existing_temporal = [col for col in temporal_features if col in df.columns]
print(f"Temporal features: {existing_temporal}")

# POI Feature Summary
print(f"\n--- POI Features Summary ---")
poi_features = [col for col in df.columns if any(keyword in col for keyword in ['POI', 'DISTANCE', 'COUNT', 'WITHIN'])]
print(f"POI-related features: {len(poi_features)}")
if poi_features:
    print(f"Sample POI features: {poi_features[:5]}{'...' if len(poi_features) > 5 else ''}")

# Data Validation Warnings
print(f"\n--- Data Validation Warnings ---")
warnings_list = []

# Check for high cardinality categorical features
high_cardinality_threshold = 50
high_cardinality_cols = [col for col, count in cardinality_info if count > high_cardinality_threshold]
if high_cardinality_cols:
    warnings_list.append(f"High cardinality categorical features (>{high_cardinality_threshold}): {high_cardinality_cols}")

# Check for features with many missing values
high_missing_threshold = 0.5
high_missing_cols = []
for col in df.columns:
    missing_pct = df[col].isnull().sum() / len(df)
    if missing_pct > high_missing_threshold:
        high_missing_cols.append((col, missing_pct))

if high_missing_cols:
    warnings_list.append(f"Features with >50% missing data: {[(col, f'{pct:.1%}') for col, pct in high_missing_cols]}")

# Check for potential data leakage (future dates)
if 'YEAR' in df.columns:
    current_year = datetime.now().year
    future_dates = (df['YEAR'] > current_year).sum()
    if future_dates > 0:
        warnings_list.append(f"Found {future_dates} records with future dates")

if warnings_list:
    for warning in warnings_list:
        print(f"  ⚠️ {warning}")
else:
    print("  ✅ No data quality warnings detected")

# Pipeline Files Summary
print(f"\n--- Generated Files ---")
files_created = {
    'Feature Engineered Dataset': os.path.join(feature_engineering_dir, "feature_engineered_crime_data.csv"),
    'Final Processed Dataset': os.path.join(feature_engineering_dir, "feature_engineered_scaled_pca_crime_data.csv")
}

for file_type, file_path in files_created.items():
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / 1024**2
        print(f"  ✅ {file_type}: {file_path} ({file_size:.2f} MB)")
    else:
        print(f"  ❌ {file_type}: {file_path} (Not found)")

print(f"\n✅ FEATURE ENGINEERING COMPLETED SUCCESSFULLY")
print(f"\n--- Next Steps for ML ---")
print("1. Load the processed dataset for modeling")
print("2. Split data into training/testing sets")
print("3. Train and evaluate machine learning models")
print("4. Address any data quality warnings if needed")